In [1]:
import pyodbc
import numpy as np
import pickle
from deepface import DeepFace
from PIL import Image
import cv2
import io
import base64
from pathlib import Path
from deepface.commons import functions, distance as dst
import sys, os
from sklearn import metrics
import pandas as pd
import timeit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [27]:
SERVER = 'localhost,1433'
DATABASE = 'bankapp'
USERNAME = 'sa'
PASSWORD = 'Hieu2301*'
sql_db = pyodbc.connect(
'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+
';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
cursor = sql_db.cursor()

In [24]:
def get_top6_each_user():
    cursor.execute('''
    SELECT face_img, user_id, rn
    FROM    
    ( SELECT face_img, user_id, id, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY id DESC) AS rn FROM dbo.face) tmp 
    WHERE rn <= 6
    ORDER BY user_id
    ''')
    return cursor.fetchall()

In [25]:
all_faces = get_top6_each_user()

In [74]:
base_img = pickle.loads(all_faces[0][0])[0, :, :, :][:, :, ::-1]
base_img *= 255
# cv2.imwrite(f"face_base.jpg", base_img_str)

In [ ]:
user_list = []
faces_arr = []
img_pair = []
user_list.append(all_faces[0][1])
for row in all_faces:
    if row[1] != user_list[-1]:
        faces_arr.append(img_pair)
        user_list.append(row[1])
        img_pair = []
    face = pickle.loads(row[0])[0, :, :, :][:, :, ::-1]
    face *= 255
    cv2.imwrite(f"db_face/face_{row[1]}_{row[2]}.jpg", face)
    img_pair.append([base_img, face])
faces_arr.append(img_pair)

In [73]:
DeepFace.verify(img1_path = base_img, img2_path = pickle.loads(all_faces[15][0])[0, :, :, :][:, :, ::-1], model_name='Facenet',  enforce_detection = False, prog_bar = False, distance_metric = 'euclidean',detector_backend = 'ssd')

1/1 [==============================] - 0s 63ms/step


{'verified': True,
 'distance': 0.004054473738085651,
 'threshold': 10,
 'model': 'Facenet',
 'detector_backend': 'ssd',
 'similarity_metric': 'euclidean'}

In [ ]:
for idx in range(len(faces_arr)):
    verified = 0
    verified_dict = DeepFace.verify(img1_path = faces_arr[idx], model_name='Facenet', distance_metric = 'cosine', enforce_detection = False, detector_backend = 'ssd', prog_bar = False)
    for key in verified_dict:
        if verified_dict[key]['verified']:
            verified += 1
    print(f'num of true face: {verified} for user: {user_list[idx]}')

In [113]:
test_img = pickle.loads(all_faces[10][0])[0, :, :, :][:, :, ::-1]
test_img *= 255
cv2.imwrite(f"face_test.jpg", test_img)

True

In [106]:
all_faces[0][1]

11

In [ ]:
num_img = 0
for row in all_faces:
    print(row[1])
    # face = pickle.loads(row[0])[0, :, :, :][:, :, ::-1]
    # face *= 255
    # cv2.imwrite(f"face_{num_img}.jpg", face)
    # num_img +=1

In [107]:
face = pickle.loads(all_faces[0][0])[0, :, :, :][:, :, ::-1]
face *= 255
cv2.imwrite(f"face_{num_img}.jpg", face)
num_img +=1

In [ ]:
all_faces[10][0]

In [97]:
cv2.imwrite('Image_from_array.jpeg', test_img)

True

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
execution_time = 0
while True:
    isSuccess, frame = cap.read()
    if isSuccess:
        try:
            user_list = []
            faces_arr = []
            img_pair = []
            user_list.append(all_faces[0][1])
            for row in all_faces:
                if row[1] != user_list[-1]:
                    faces_arr.append(img_pair)
                    user_list.append(row[1])
                    img_pair = []
                face = pickle.loads(row[0])[0, :, :, :][:, :, ::-1]
                face *= 255
                img_pair.append([frame, face])
            faces_arr.append(img_pair)
            for idx in range(len(faces_arr)):
                verified = 0
                verified_dict = DeepFace.verify(img1_path = faces_arr[idx], model_name='Facenet', distance_metric = 'cosine', enforce_detection = False, detector_backend = 'ssd')
                for key in verified_dict:
                    if verified_dict[key]['verified']:
                        verified += 1
                print(f'num of true face: {verified} for user: {user_list[idx]}')
            break
        except Exception as e:
            print(e)
            continue
cap.release()

In [105]:
def verify(img_1, img_2, model_name, distance_metric):

	img1_embedding_obj = DeepFace.represent(img_path = img_1
							, model_name = model_name
							, enforce_detection = False
							, detector_backend = "skip"
							, align = True
							, normalization = 'Facenet'
							)
				
	img2_embedding_obj = DeepFace.represent(img_path = img_2
							, model_name = model_name
							, enforce_detection = False
							, detector_backend = "skip"
							, align = True
							, normalization = 'Facenet'
							)
				
	img1_representation = img1_embedding_obj[0]["embedding"]
	img2_representation = img2_embedding_obj[0]["embedding"]

	if distance_metric == 'cosine':
		distance = dst.findCosineDistance(img1_representation, img2_representation)
	elif distance_metric == 'euclidean':
		distance = dst.findEuclideanDistance(img1_representation, img2_representation)
	elif distance_metric == 'euclidean_l2':
		distance = dst.findEuclideanDistance(dst.l2_normalize(img1_representation), dst.l2_normalize(img2_representation))
	else:
		raise ValueError("Invalid distance_metric passed - ", distance_metric)
	
	threshold = dst.findThreshold(model_name, distance_metric)

	return True if distance <= threshold else False


In [20]:
base_img = pickle.loads(all_faces[0][0])[0, :, :, :]
img_2 = pickle.loads(all_faces[10][0])

In [113]:
face_1 = functions.extract_faces(img= 'D:\Code\\thesis\\face-service\data\\btLinh_0.jpg',target_size = (160, 160))
face_2 = functions.extract_faces(img= 'D:\Code\\thesis\\face-service\data\\hpQuyen_1.jpg',target_size = (160, 160))
verify(face_1[0][0], face_2[0][0], 'Facenet', 'cosine')

False

In [3]:
file_list = []
data_path = 'D:\Code\\thesis\\face-service\data'
for file in os.listdir(data_path):
    if file.endswith(".jpg"):
        file_list.append(os.path.join(data_path, file))

In [168]:
representation_list = []
for file in file_list:
    user_name = file.replace('D:\Code\\thesis\\face-service\\data\\', '').split('_')[0]
    face = functions.extract_faces(img= file, target_size = (150, 150), detector_backend = 'ssd', enforce_detection = False)
    embedding_obj = DeepFace.represent(img_path = face[0][0]
                        , model_name = 'Dlib'
                        , enforce_detection = False
                        , detector_backend = "skip"
                        , align = True
                        , normalization = 'base'
                        )
    representation_list.append([embedding_obj[0]["embedding"], user_name])

pred_list = []
threshold = dst.findThreshold('Dlib', 'cosine')
for rps_1 in representation_list:
    for rps_2 in representation_list:
        distance = dst.findCosineDistance(rps_1[0], rps_2[0])
        pred = True if distance <= threshold else False 
        pred_list.append(pred)

In [139]:
test_list = []
for rps_1 in representation_list:
    for rps_2 in representation_list:
        test = True if rps_1[1] == rps_2[1] else False
        test_list.append(test)

In [169]:
count = 0
for idx in range(len(pred_list)):
    if pred_list[idx] == test_list[idx]:
        count+=1
count/len(pred_list)

0.15614082235958077

In [4]:
test_list = []
for file_1 in file_list:
    for file_2 in file_list:
        name_1 = file_1.replace('D:\Code\\thesis\\face-service\\data\\', '').split('_')[0]
        name_2 = file_2.replace('D:\Code\\thesis\\face-service\\data\\', '').split('_')[0]
        test = True if name_1 == name_2 else False
        test_list.append(test)

In [6]:
def recognition(model_name, distance_metric, detector, normalization):
    representation_list = []
    target_size = functions.find_target_size(model_name=model_name)
    for file in file_list:
        user_name = file.replace('D:\Code\\thesis\\face-service\\data\\', '').split('_')[0]
        face = functions.extract_faces(img= file, target_size = target_size, detector_backend = detector, enforce_detection = False)
        embedding_obj = DeepFace.represent(img_path = face[0][0]
                            , model_name = model_name
                            , enforce_detection = False
                            , detector_backend = "skip"
                            , align = True
                            , normalization = normalization
                            )
        representation_list.append([embedding_obj[0]["embedding"], user_name])

    pred_list = []
    threshold = dst.findThreshold(model_name, distance_metric)
    for rps_1 in representation_list:
        for rps_2 in representation_list:
            if distance_metric == 'cosine':
                distance = dst.findCosineDistance(rps_1[0], rps_2[0])
            elif distance_metric == 'euclidean':
                distance = dst.findEuclideanDistance(rps_1[0], rps_2[0])
            elif distance_metric == 'euclidean_l2':
                distance = dst.findEuclideanDistance(dst.l2_normalize(rps_1[0]), dst.l2_normalize(rps_2[0]))
            else:
                raise ValueError("Invalid distance_metric passed - ", distance_metric)
            pred = True if distance <= threshold else False 
            pred_list.append(pred)
    return pred_list

In [7]:
detectors = ['retinaface', 'mtcnn', 'ssd', 'dlib']
recognitors = ['Facenet', 'Facenet512', 'Dlib', 'ArcFace']
metrics_list = ['cosine', 'euclidean', 'euclidean_l2']

In [8]:
normalization = ''
metr_list = []
for metric in metrics_list:
    detect_list = []
    for detector in detectors:
        recog_list = []
        for recognitor in recognitors:
            if recognitor == 'Facenet' or recognitor == 'Facenet512':
                normalization = 'Facenet'
            elif recognitor == 'ArcFace':
                normalization = 'ArcFace'
            else:
                normalization = 'base'
            print(f"recognizing with {recognitor} - {detector} - {metric}")

            start = timeit.default_timer()
            pred_list = recognition(recognitor, metric, detector, normalization)

            execution_time = timeit.default_timer() - start
            accuracy = accuracy_score(test_list, pred_list)
            precision = precision_score(test_list, pred_list)
            recall = recall_score(test_list, pred_list)
            f1 = f1_score(test_list, pred_list)
            recog_list.append([accuracy, precision, recall, f1, execution_time])
        detect_list.append(recog_list)
    metr_list.append(detect_list)

recognizing with Facenet - retinaface - cosine
recognizing with Facenet512 - retinaface - cosine
recognizing with Dlib - retinaface - cosine
recognizing with ArcFace - retinaface - cosine
recognizing with Facenet - mtcnn - cosine
1/1 [==============================] - 0s 26ms/step
recognizing with Facenet512 - mtcnn - cosine
1/1 [==============================] - 0s 29ms/step
recognizing with Dlib - mtcnn - cosine
1/1 [==============================] - 0s 32ms/step
recognizing with ArcFace - mtcnn - cosine
1/1 [==============================] - 0s 32ms/step
recognizing with Facenet - ssd - cosine
recognizing with Facenet512 - ssd - cosine
recognizing with Dlib - ssd - cosine
recognizing with ArcFace - ssd - cosine
recognizing with Facenet - dlib - cosine
recognizing with Facenet512 - dlib - cosine
recognizing with Dlib - dlib - cosine
recognizing with ArcFace - dlib - cosine
recognizing with Facenet - retinaface - euclidean
recognizing with Facenet512 - retinaface - euclidean
recognizi

In [9]:
metr_list

[[[[0.8538027411986026,
    0.5084577114427861,
    0.910873440285205,
    0.6526181353767561,
    126.43769309999999],
   [0.9435635581832841,
    0.948849104859335,
    0.661319073083779,
    0.7794117647058825,
    115.20184570000004],
   [0.8177909164203171,
    0.44412607449856734,
    0.8288770053475936,
    0.5783582089552238,
    120.02794360000007],
   [0.8554152109647944,
    0.5114200595829196,
    0.9180035650623886,
    0.6568877551020408,
    111.64628600000003]],
  [[0.8769148078473529,
    0.5575418994413408,
    0.8894830659536542,
    0.6854395604395604,
    103.33557999999994],
   [0.9473259876377318,
    0.9418886198547215,
    0.6934046345811051,
    0.7987679671457906,
    101.57860110000001],
   [0.7973662993818866,
    0.4143744454303461,
    0.8324420677361853,
    0.5533175355450237,
    119.12118259999988],
   [0.8618650900295619,
    0.5236180904522613,
    0.928698752228164,
    0.6696658097686375,
    112.13895079999998]],
  [[0.8559527008868584,
    0.522

In [20]:
acc_list = []
time_list = []
for detector in metr_list[0]:
    recog_list = []
    for recog in detector:
        recog_list.append(recog[0])
    acc_list.append(recog_list)
    recog_list = []
    for recog in detector:
        recog_list.append(recog[4])
    time_list.append(recog_list)

acc_df = pd.DataFrame(acc_list, detectors, recognitors)
time_df = pd.DataFrame(time_list, detectors, recognitors)

In [69]:
prec_list = []
rec_list = []
for detector in metr_list[2]:
    recog_list = []
    for recog in detector:
        recog_list.append(recog[1])
    prec_list.append(recog_list)

pred_df = pd.DataFrame(prec_list, detectors, recognitors)

In [70]:
pred_df

,Facenet,Facenet512,Dlib,ArcFace
retinaface,0.691002,0.450283,0.350105,0.608323
mtcnn,0.745063,0.438541,0.327261,0.617458
ssd,0.632047,0.441753,0.262064,0.431847
dlib,0.851852,0.471778,0.442654,0.618209


In [66]:
rec_df

,Facenet,Facenet512,Dlib,ArcFace
retinaface,0.910873,0.661319,0.828877,0.918004
mtcnn,0.889483,0.693405,0.832442,0.928699
ssd,0.522282,0.379679,0.572193,0.636364
dlib,0.739750,0.422460,0.811052,0.800357


In [98]:
acc_list = []
time_list = []
prec_list = []
rec_list = []
f1_list = []
for metric in metr_list:
    recog_list = []
    for recog in metric[3]:
        recog_list.append(recog[0])
    acc_list.append(recog_list)
    recog_list = []
    for recog in metric[3]:
        recog_list.append(recog[1])
    prec_list.append(recog_list)
    recog_list = []
    for recog in metric[3]:
        recog_list.append(recog[2])
    rec_list.append(recog_list)
    recog_list = []
    for recog in metric[3]:
        recog_list.append(recog[3])
    f1_list.append(recog_list)
    recog_list = []
    for recog in metric[3]:
        recog_list.append(recog[4])
    time_list.append(recog_list)

In [99]:
acc_df = pd.DataFrame(acc_list, metrics_list, recognitors)
prec_df = pd.DataFrame(prec_list, metrics_list, recognitors)
rec_df = pd.DataFrame(rec_list, metrics_list, recognitors)
f1_df = pd.DataFrame(f1_list, metrics_list, recognitors)
time_df = pd.DataFrame(time_list, metrics_list, recognitors)

In [182]:
obj = embedding_obj = DeepFace.represent(img_path = 'D:\Code\\thesis\\face-service\data\\btLinh_0.jpg'
                    , model_name = 'Facenet'
                    , detector_backend = "ssd"
                    , align = True
                    , normalization = 'Facenet'
                    )

In [184]:
DeepFace.analyze(img_path = 'D:\Code\\thesis\\face-service\data\\btLinh_0.jpg', actions = ('emotion'))

Action: emotion: 100%|██████████| 1/1 [00:12<00:00, 12.27s/it]


[{'emotion': {'angry': 0.6117865179584601,
   'disgust': 0.0009242732438484903,
   'fear': 53.9025421258757,
   'happy': 6.096117216811512,
   'sad': 1.2610411630819842,
   'surprise': 31.541755015640852,
   'neutral': 6.585825194618082},
  'dominant_emotion': 'fear',
  'region': {'x': 524, 'y': 880, 'w': 305, 'h': 305}}]

In [5]:
METRIC = 'cosine'
def compute_distance(representation_1, representation_2):
    if METRIC == 'cosine':
        return dst.findCosineDistance(representation_1, representation_2)
    elif METRIC == 'euclidean':
        return dst.findEuclideanDistance(representation_1, representation_2)
    elif METRIC == 'euclidean_l2':
        return dst.findEuclideanDistance(dst.l2_normalize(representation_1), dst.l2_normalize(representation_2))
    else:
        raise ValueError("Invalid distance_metric passed - ", METRIC)

In [33]:
embedding_obj = DeepFace.represent(img_path = face[0][0]
                    , model_name = 'Facenet'
                    , enforce_detection = False
                    , detector_backend = "skip"
                    , align = True
                    , normalization = 'Facenet'
                    )

In [ ]:
cam = cv2.VideoCapture(0)
result, image = cam.read()
  
# If image will detected without any error, 
# show result
if result:
  
    # showing result, it take frame name and image 
    # output
    cv2.imshow("GeeksForGeeks", image)
  
    # saving image in local storage
    cv2.imwrite("GeeksForGeeks.png", image)
  
    # If keyboard interrupt occurs, destroy image 
    # window
    cv2.waitKey(0)
    cv2.destroyWindow("GeeksForGeeks")
  
# If captured image is corrupted, moving to else part
else:
    print("No image detected. Please! try again")

In [39]:
target_size = functions.find_target_size(model_name='Facenet512')

In [41]:
face = functions.extract_faces(img= "D:\Code\\thesis\\face-service\src\\feature\GeeksForGeeks.png", target_size = target_size, detector_backend = 'mtcnn')

1/1 [==============================] - 1s 1s/step


In [51]:
img = face[0][0][0]
cv2.imwrite('CroppedFace.jpeg', img*255)

In [54]:
face[0][1]

{'x': 307, 'y': 105, 'w': 149, 'h': 183}

In [57]:
embedding = DeepFace.represent(img_path = face[0][0]
                            , model_name = 'Facenet512'
                            , enforce_detection = False
                            , detector_backend = "skip"
                            , align = True
                            , normalization = 'Facenet'
                            )

In [60]:
embedding[0]['embedding']

[1.6530414819717407,
 -0.33907559514045715,
 -0.2434367537498474,
 2.075732946395874,
 -0.3729766607284546,
 0.33439600467681885,
 -0.8160080909729004,
 0.29653024673461914,
 1.495720386505127,
 0.18276745080947876,
 0.3023906350135803,
 2.6115689277648926,
 0.2947551906108856,
 1.4504592418670654,
 -0.04924469441175461,
 -0.3709430694580078,
 1.8076554536819458,
 0.9304078221321106,
 -0.3546365797519684,
 1.1814790964126587,
 -0.4233786165714264,
 0.6523009538650513,
 2.5056631565093994,
 -1.2855809926986694,
 -0.3469253480434418,
 -0.14459365606307983,
 0.4323810338973999,
 -0.05099127069115639,
 0.14454585313796997,
 -0.37606358528137207,
 0.7865155339241028,
 0.4865807890892029,
 -1.255374789237976,
 0.06779079139232635,
 -0.5030909180641174,
 -0.8320950865745544,
 -1.6104040145874023,
 -0.9711840152740479,
 -1.8172473907470703,
 0.5165275931358337,
 0.7281861901283264,
 -1.0333305597305298,
 -1.2291909456253052,
 -0.6421117186546326,
 -1.4842896461486816,
 -0.18768613040447235,
 0

In [61]:
analysis = DeepFace.analyze(img_path = 'D:\Code\\thesis\\face-service\src\\feature\GeeksForGeeks.png', 
                                        actions = ('emotion'), 
                                        detector_backend = 'ssd', 
                                        silent=True)

In [62]:
analysis

[{'emotion': {'angry': 70.22009910464878,
   'disgust': 1.7119787497756437e-06,
   'fear': 0.2580734123631366,
   'happy': 0.0026207544322039778,
   'sad': 3.0292056443813826,
   'surprise': 0.0007349810386292277,
   'neutral': 26.489265351847727},
  'dominant_emotion': 'angry',
  'region': {'x': 307, 'y': 100, 'w': 153, 'h': 184}}]